### Why?
Traditional retrieval methods return isolated text chunks, which can lead to incomplete answers. 

To address this, I introduce Context-Enriched Retrieval, which ensures that retrieved information includes neighboring chunks for better coherence.

#### Steps in This Notebook:

- Data Ingestion: Extract text from a PDF.
- Chunking with Overlapping Context: Split text into overlapping chunks to preserve context.
- Embedding Creation: Convert text chunks into numerical representations.
- Context-Aware Retrieval: Retrieve relevant chunks along with their neighbors for better completeness.
- Response Generation: Use a language model to generate responses based on retrieved context.
- Evaluation: Assess the model's response accuracy.


In [2]:
import fitz
import os
from dotenv import load_dotenv
import numpy as np
import json
from openai import OpenAI

from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
def extract_text_from_pdf(pdf_path):
    pdf = fitz.open(pdf_path)
    all_text = ""

    for page in pdf:
        all_text += page.get_text("text")
    
    return all_text

pdf_path = "../data/AI_Information.pdf"

extracted_text = extract_text_from_pdf(pdf_path)

In [4]:
extracted_text

'Understanding Artificial Intelligence \nChapter 1: Introduction to Artificial Intelligence \nArtificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot \nto perform tasks commonly associated with intelligent beings. The term is frequently applied to \nthe project of developing systems endowed with the intellectual processes characteristic of \nhumans, such as the ability to reason, discover meaning, generalize, or learn from past \nexperience. Over the past few decades, advancements in computing power and data availability \nhave significantly accelerated the development and deployment of AI. \nHistorical Context \nThe idea of artificial intelligence has existed for centuries, often depicted in myths and fiction. \nHowever, the formal field of AI research began in the mid-20th century. The Dartmouth Workshop \nin 1956 is widely considered the birthplace of AI. Early AI research focused on problem-solving \nand symbolic methods. The 1980s saw 

In [ ]:
def chunk_text(text, n, overlap):
    return [text[i:i+n] for i in range(0, len(text), n - overlap)]

text_chunks = chunk_text(extracted_text, 1000, 200)

['Understanding Artificial Intelligence \nChapter 1: Introduction to Artificial Intelligence \nArtificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot \nto perform tasks commonly associated with intelligent beings. The term is frequently applied to \nthe project of developing systems endowed with the intellectual processes characteristic of \nhumans, such as the ability to reason, discover meaning, generalize, or learn from past \nexperience. Over the past few decades, advancements in computing power and data availability \nhave significantly accelerated the development and deployment of AI. \nHistorical Context \nThe idea of artificial intelligence has existed for centuries, often depicted in myths and fiction. \nHowever, the formal field of AI research began in the mid-20th century. The Dartmouth Workshop \nin 1956 is widely considered the birthplace of AI. Early AI research focused on problem-solving \nand symbolic methods. The 1980s saw

In [6]:
load_dotenv("../conf.env")
client = OpenAI(
    api_key=os.getenv("GEMINI_API_KEY"),
    base_url=os.getenv("GEMINI_BASE_URL")
)

In [9]:
len(text_chunks), text_chunks[0]

(42,
 'Understanding Artificial Intelligence \nChapter 1: Introduction to Artificial Intelligence \nArtificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot \nto perform tasks commonly associated with intelligent beings. The term is frequently applied to \nthe project of developing systems endowed with the intellectual processes characteristic of \nhumans, such as the ability to reason, discover meaning, generalize, or learn from past \nexperience. Over the past few decades, advancements in computing power and data availability \nhave significantly accelerated the development and deployment of AI. \nHistorical Context \nThe idea of artificial intelligence has existed for centuries, often depicted in myths and fiction. \nHowever, the formal field of AI research began in the mid-20th century. The Dartmouth Workshop \nin 1956 is widely considered the birthplace of AI. Early AI research focused on problem-solving \nand symbolic methods. The 1980

### Create embeddings

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(os.getenv("LOCAL_EMBED_EN")).to(device)
tokenizer = AutoTokenizer.from_pretrained(os.getenv("LOCAL_EMBED_EN"))

def embed(text): # return numpy.ndarray(n, x) 
    if isinstance(text, str): text = [text] # if single string => convert into a list of one element
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(device) # tokenize input
    with torch.no_grad(): 
        output = model(**inputs) # running model
        embedding = F.normalize(output.last_hidden_state[:, 0, :], p=2, dim=1) # normalize vector to L2
    return embedding.cpu().numpy() # pass to cpu with numpy array

In [24]:
response = embed(text_chunks)
response.shape, type(response)

((42, 768), numpy.ndarray)

In [46]:
response[0].shape # (x,)

(768,)

### Implementing Context-Aware Semantic Search

In [ ]:
def context_enriched_search(query, text_chunks, embeddings, k=1, context_size=1):
    """
    Retrieves the most relevant chunk along with its neighboring chunks.

    Args:
    query (str): Search query.
    text_chunks (List[str]): List of text chunks.
    embeddings (List[dict]): List of chunk embeddings.
    k (int): Number of relevant chunks to retrieve.
    context_size (int): Number of neighboring chunks to include.

    Returns:
    List[str]: Relevant text chunks with contextual information.
    """
    # Convert the query into an embedding vector
    query_embedding = embed(query)
    similarity_scores = [(i, cosine_similarity(query_embedding, embs.reshape(1, -1))[0][0]) 
                         for i, embs in enumerate(embeddings)] #[[i, numpy.ndarray(float)],[],[],...]
    print(similarity_scores[:5])
    # Sort chunks by similarity score in descending order (highest similarity first)
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    print(similarity_scores[:5])
    # Get the index of the most relevant chunk
    top_index = similarity_scores[0][0]
    # Define the range for context inclusion
    # Ensure we don't go below 0 or beyond the length of text_chunks
    start = max(0, top_index - context_size)
    end = min(len(text_chunks), top_index + context_size + 1)
    print(start, end)
    # Return the relevant chunk along with its neighboring context chunks
    return [text_chunks[i] for i in range(start, end)]


In [76]:
# Load the validation dataset from a JSON file
with open('../data/val.json') as f:
    data = json.load(f)

# Extract the first question from the dataset to use as our query
query = data[0]['question']

# Retrieve the most relevant chunk and its neighboring chunks for context
# Parameters:
# - query: The question we're searching for
# - text_chunks: Our text chunks extracted from the PDF
# - response.data: The embeddings of our text chunks
# - k=1: Return the top match
# - context_size=1: Include 1 chunk before and after the top match for context
top_chunks = context_enriched_search(query, text_chunks, response, k=1, context_size=1)

# Print the query for reference
print("Query:", query)
# Print each retrieved chunk with a heading and separator
for i, chunk in enumerate(top_chunks):
    print(f"Context {i + 1}:\n{chunk}\n=====================================")

[(0, np.float32(0.8246176)), (1, np.float32(0.79339814)), (2, np.float32(0.81529665)), (3, np.float32(0.7903184)), (4, np.float32(0.78320646))]
[(37, np.float32(0.85300636)), (8, np.float32(0.8340821)), (0, np.float32(0.8246176)), (29, np.float32(0.8236108)), (36, np.float32(0.8229671))]
36 39
Query: What is 'Explainable AI' and why is it considered important?
Context 1:
nt aligns with societal values. Education and awareness campaigns inform the public 
about AI, its impacts, and its potential. 
Chapter 19: AI and Ethics 
Principles of Ethical AI 
Ethical AI principles guide the development and deployment of AI systems to ensure they are fair, 
transparent, accountable, and beneficial to society. Key principles include respect for human 
rights, privacy, non-discrimination, and beneficence. 
 
 
Addressing Bias in AI 
AI systems can inherit and amplify biases present in the data they are trained on, leading to unfair 
or discriminatory outcomes. Addressing bias requires careful data c

In [78]:
# Define the system prompt for the AI assistant
system_prompt = "You are an AI assistant that strictly answers based on the given context. If the answer cannot be derived directly from the provided context, respond with: 'I do not have enough information to answer that.'"

def generate_response(system_prompt, user_message, model=os.getenv("GEMINI_GEN_MODEL")):
    """
    Generates a response from the AI model based on the system prompt and user message.

    Args:
    system_prompt (str): The system prompt to guide the AI's behavior.
    user_message (str): The user's message or query.
    model (str): The model to be used for generating the response. Default is "meta-llama/Llama-2-7B-chat-hf".

    Returns:
    dict: The response from the AI model.
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# Create the user prompt based on the top chunks
user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt = f"{user_prompt}\nQuestion: {query}"

# Generate AI response
ai_response = generate_response(system_prompt, user_prompt)


In [79]:
ai_response_text = ai_response.choices[0].message.content
ai_response_text

'Explainable AI (XAI) techniques aim to make AI decisions more understandable. It is considered important because transparency and explainability are essential for building trust in AI systems, enabling users to assess their fairness, accuracy, and reliability.'

In [80]:
# Define the system prompt for the evaluation system
evaluate_system_prompt = "You are an intelligent evaluation system tasked with assessing the AI assistant's responses. If the AI assistant's response is very close to the true response, assign a score of 1. If the response is incorrect or unsatisfactory in relation to the true response, assign a score of 0. If the response is partially aligned with the true response, assign a score of 0.5."

# Create the evaluation prompt by combining the user query, AI response, true response, and evaluation system prompt
evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response_text}\nTrue Response: {data[0]['ideal_answer']}\n{evaluate_system_prompt}"

# Generate the evaluation response using the evaluation system prompt and evaluation prompt
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# Print the evaluation response
print(evaluation_response.choices[0].message.content)

Score: 1


In [81]:
ai_res = embed(ai_response_text)
ideal_res = embed(data[0]['ideal_answer'])

print(cosine_similarity(ai_res, ideal_res))

[[0.9802352]]
